In [1]:
import pandas as pd
import os

caminho = 'C:\\Users\\Edmilson\\Gama_Workspace2\\ABMS-WP'

# 1. Carregar os dados
# Carregar o Tabela_consumidores_Itapua_com_setor.csv com SK_MATRICULA e NN_MORADORES
arquivo_df = pd.read_csv(os.path.join(caminho,'includes\\Tabela_consumidores_Itapua_com_setor.csv'), sep=',')

# Carregar o arquivo Tabela_consumo_Itapua_12m.csv
consumo_df = pd.read_csv(os.path.join(caminho,'includes\\Tabela_consumo_Itapua_60m.csv'), sep=';')

# 2. Calcular a média de consumo dos últimos 6 meses para cada SK_MATRICULA
# Converter AM_REFERENCIA para datetime para facilitar a filtragem
consumo_df['AM_REFERENCIA'] = pd.to_datetime(consumo_df['AM_REFERENCIA'], format='%Y%m')

# Ordenar por SK_MATRICULA e AM_REFERENCIA
consumo_df = consumo_df.sort_values(by=['SK_MATRICULA', 'AM_REFERENCIA'], ascending=[True, False])

# Filtrar os últimos 12 meses para cada SK_MATRICULA
ultimos_12_meses = consumo_df.groupby('SK_MATRICULA').head(12)

# Calcular a média de consumo (NN_MEDIA_CONSUMO) para cada SK_MATRICULA
media_consumo = ultimos_12_meses.groupby('SK_MATRICULA')['HCLQTCON'].mean().reset_index()
media_consumo.rename(columns={'HCLQTCON': 'NN_MEDIA_CONSUMO'}, inplace=True)


# 3. Juntar a média de consumo ao arquivo.csv
arquivo_df = pd.merge(arquivo_df, media_consumo, on='SK_MATRICULA', how='left')

# Primeiro, remover registros onde CD_SETOR é nulo ou vazio
arquivo_df = arquivo_df.dropna(subset=['CD_SETOR'])  # Remove registros com CD_SETOR nulo
arquivo_df = arquivo_df[arquivo_df['CD_SETOR'].astype(str) != '']  # Remove registros com CD_SETOR vazio

# Extrair os últimos 3 caracteres da coluna CD_SETOR
arquivo_df['CD_SETOR'] = 'S' + arquivo_df['CD_SETOR'].astype(str).str[-3:]

# Tratar casos onde NN_MORADORES == 0, substituindo por 1
arquivo_df['NN_MORADORES'] = arquivo_df['NN_MORADORES'].replace(0, 1)

# 4. Classificar o comportamento (AMBIENTALISTA, NORMAL ou PERDULÁRIO)
# AMBIENTALISTA < 130
# NORMAL > 130 E <= 160
# PERDULARIO > 160
arquivo_df['NN_CONSUMO_DIARIO'] = ((arquivo_df['NN_MEDIA_CONSUMO']*1000 / arquivo_df['NN_MORADORES']) / 30.5)
arquivo_df['TP_COMPORTAMENTO'] = arquivo_df['NN_CONSUMO_DIARIO'].apply(
    lambda consumo: 
                   'AMBIENTALISTA' if consumo < 130 
                   else 'MODERADO' if 130 <= consumo <= 160 
                   else 'PERDULARIO'
)


# 5. Salvar o arquivo.csv com a nova coluna TP_COMPORTAMENTO
arquivo_df.to_csv(os.path.join(caminho,'includes\\Tabela_consumidores_Itapua_com_setor_e_comportamento.csv'), sep=';',index=False)

# 6. Visualizar o resultado
print(arquivo_df.head())

# Calcular estatísticas por perfil
contagem_comportamento = arquivo_df['TP_COMPORTAMENTO'].value_counts()
percentual_comportamento = arquivo_df['TP_COMPORTAMENTO'].value_counts(normalize=True) * 100
media_consumo_perfil = arquivo_df.groupby('TP_COMPORTAMENTO')['NN_CONSUMO_DIARIO'].mean()



                                        SK_MATRICULA NM_LOCALIDADE  \
0  6E60FF997A2A7FD7665AC40BAD9BE286D7C47A1C3521CE...  SALVADOR UMB   
1  E6195B23791D174F7C612B6324CACDED0CBDC2283FC728...  SALVADOR UMB   
2  0C2ED22E52F7B889C5DCB715B44E40C0CF4E25A77F1A17...  SALVADOR UMB   
3  6D38EE4CEC85F41B626000179F50D3599EEFA9B20AC328...  SALVADOR UMB   
4  0420BAE41A9CA4FBA6CE1F1E120C73D05448330C834D72...  SALVADOR UMB   

  NM_CATEGORIATARIFARIA NM_SITUACAO_IMOVEL  NN_MORADORES  ST_PISCINA  \
0           RESIDENCIAL           HABITADO             1           0   
1           RESIDENCIAL           HABITADO             1           0   
2           RESIDENCIAL           HABITADO             1           0   
3           RESIDENCIAL           HABITADO             1           0   
4           RESIDENCIAL           HABITADO             1           0   

     LAT_GEO   LONG_GEO              X             Y CD_SETOR  \
0 -12.938243 -38.359220  569505.509777  8.569607e+06     S166   
1 -12.934564 -38

In [2]:
# Combinar os resultados em um único DataFrame
resultado = pd.DataFrame({
    'Contagem': contagem_comportamento,
    'Percentual (%)': percentual_comportamento,
    'Média Consumo Diário (L)': media_consumo_perfil
})

# Exibir o resultado
print("\nEstatísticas por Perfil de Consumo:")
print(resultado)


Estatísticas por Perfil de Consumo:
                  Contagem  Percentual (%)  Média Consumo Diário (L)
TP_COMPORTAMENTO                                                    
AMBIENTALISTA         7965       52.069033                 52.621962
MODERADO              1086        7.099431                144.315598
PERDULARIO            6246       40.831536                510.352010
